In [ ]:
import requests
from requests import JSONDecodeError        #用來處理當下載資料非JSON檔時
from pprint import pprint                   #用來美化列印

aqi_url="https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=JSON"
try:
    response=requests.get(aqi_url)
except Exception as Error:                  #出錯時執行
    print(Error)
else:
    if response.status_code==200:           #確認資料正確下載
        try:
            all_data:dict[any]=response.json()  #從json轉換成python資料結構
            pprint(all_data)
        except JSONDecodeError:             #當格式非JSON時
            print("api_key為測試用，連線已至上限，請稍後再試")
    else:
        print("下載狀態碼不是200")



In [3]:
#由於巢狀敘述太多層，將其變為function，此作法被稱為逃脫
import requests
from requests import JSONDecodeError 
from pprint import pprint
def download_json()->dict[any]:    #已知回傳為python的dict結構
    aqi_url="https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=JSON"
    try:
        response=requests.get(aqi_url)
    except Exception:
        raise Exception("連線失敗") #回傳錯誤狀態
    else:
        if response.status_code==200:   #200代表OK
            try:
                all_data:dict[any]=response.json()
                return all_data     #從print改成return回傳資料
            except JSONDecodeError:
                raise Exception("api_key為測試用，連線已至上限，請稍後再試")
        else:
            raise Exception("下載狀態碼不是200")

In [4]:
try:
    all_data:dict[any]=download_json()
    print(all_data['records'])             #利用[中括號]指定key為records的資料
except Exception as error:
    print(error)

[{'sitename': '屏東(枋山)', 'county': '屏東縣', 'aqi': '51', 'pollutant': '臭氧八小時', 'status': '普通', 'so2': '0.1', 'co': '0.14', 'o3': '55.4', 'o3_8hr': '55.5', 'pm10': '23', 'pm2.5': '8', 'no2': '1.8', 'nox': '2', 'no': '0.1', 'windspeed': '4.4', 'winddirec': '86', 'datacreationdate': '2024-06-05 10:00', 'unit': '', 'co_8hr': '0.1', 'pm2.5_avg': '9.4', 'pm10_avg': '25', 'so2_avg': '0', 'longitude': '120.651472', 'latitude': '22.260899', 'siteid': '313'}, {'sitename': '大甲（日南國小）', 'county': '臺中市', 'aqi': '28', 'pollutant': '', 'status': '良好', 'so2': '0.4', 'co': '0.42', 'o3': '31.2', 'o3_8hr': '24.3', 'pm10': '11', 'pm2.5': '5', 'no2': '9.3', 'nox': '10.9', 'no': '1.5', 'windspeed': '1.6', 'winddirec': '47', 'datacreationdate': '2024-06-05 10:00', 'unit': '', 'co_8hr': '0.3', 'pm2.5_avg': '8.5', 'pm10_avg': '12', 'so2_avg': '0', 'longitude': '120.65525382', 'latitude': '24.39017512', 'siteid': '312'}, {'sitename': '新北(樹林)', 'county': '新北市', 'aqi': '64', 'pollutant': '細懸浮微粒', 'status': '普通', 'so2

In [5]:
#資料中的records欄位為list，其中包含了dict
#利用RootModel，讓list可以應用其內的dict結構
from pydantic import BaseModel,Field,RootModel

class Site(BaseModel):      #定義list裡面的dict名為Site
    site_name:str=Field(alias='sitename')
    county:str
    aqi:int
    status:str

class Records(RootModel):
    root:list[Site]

records:Records=Records.model_validate(all_data['records']) 
records.model_dump()

[{'site_name': '屏東(枋山)', 'county': '屏東縣', 'aqi': 51, 'status': '普通'},
 {'site_name': '大甲（日南國小）', 'county': '臺中市', 'aqi': 28, 'status': '良好'},
 {'site_name': '新北(樹林)', 'county': '新北市', 'aqi': 64, 'status': '普通'},
 {'site_name': '屏東（琉球）', 'county': '屏東縣', 'aqi': 30, 'status': '良好'},
 {'site_name': '臺南（麻豆）', 'county': '臺南市', 'aqi': 35, 'status': '良好'},
 {'site_name': '高雄（湖內）', 'county': '高雄市', 'aqi': 19, 'status': '良好'},
 {'site_name': '彰化（員林）', 'county': '彰化縣', 'aqi': 44, 'status': '良好'},
 {'site_name': '大城', 'county': '彰化縣', 'aqi': 41, 'status': '良好'},
 {'site_name': '富貴角', 'county': '新北市', 'aqi': 51, 'status': '普通'},
 {'site_name': '麥寮', 'county': '雲林縣', 'aqi': 45, 'status': '良好'},
 {'site_name': '關山', 'county': '臺東縣', 'aqi': 30, 'status': '良好'},
 {'site_name': '馬公', 'county': '澎湖縣', 'aqi': 47, 'status': '良好'},
 {'site_name': '金門', 'county': '金門縣', 'aqi': 45, 'status': '良好'},
 {'site_name': '馬祖', 'county': '連江縣', 'aqi': 51, 'status': '普通'},
 {'site_name': '埔里', 'county': '南投縣', 'aqi': 

In [ ]:
from pydantic import BaseModel,Field,RootModel,field_validator

class Site(BaseModel):      #定義list裡面的dict名為Site
    site_name:str=Field(alias="sitename")
    county:str
    aqi:int
    status:str
    pm25:float =Field(alias='pm2.5')
    date:str=Field(alias='datacreationdate')

    @field_validator("pm25",mode='before')  #事先對字串做判斷，防止其內的資料為空
    @classmethod
    def abc(cls,value:str)->str:
        if value=="":
            return "0.0"
        else:
            return value

class Records(RootModel):
    root:list[Site]

records:Records=Records.model_validate(all_data['records']) 
data=records.model_dump()
data

In [ ]:
def main():
    pass

if __name__=='__main__':
    main()